<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2024/blob/main/IMT_PCA_scikitlearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **PCA: Principal Component Analysis**

O **PCA (Principal Component Analysis)**, ou Análise de Componentes Principais, é uma técnica matemática criada em 1901 por Karl Pearson. Ela transforma observações de variáveis possivelmente correlacionadas em valores de variáveis linearmente não correlacionadas chamados componentes principais. Os componentes principais são independentes apenas se os dados forem normalmente distribuídos conjuntamente.

A técnica é sensível à escala das variáveis originais e é frequentemente usada na análise exploratória de dados e na construção de modelos preditivos. O PCA é realizado por meio da decomposição em autovalores de uma matriz de covariância após a centralização e normalização dos dados. Os resultados são expressos em termos de pontuações de componentes, fornecendo uma representação em dimensões mais baixas dos dados originais, revelando sua estrutura interna, de uma forma que melhor explica a variância nos dados. Na visualização de dados, o PCA ainda pode ainda fornecer uma visualização em dimensões mais baixas dos dados, com uma verdadeira *sombra* do objeto original quando visto de seu ponto mais informativo.

O PCA é, assim, uma técnica de aprendizado não supervisionado, frequentemente usada para descobrir estruturas em dados de alta dimensão e para redução de dimensionalidade.


## PCA Layman's Introduction

Antes de entrarmos na matemática e no uso dessa técnica entenda esse importante conceito com esse vídeo de 1.33min:

In [2]:
from IPython.display import YouTubeVideo
video = YouTubeVideo('BfTMmoDFXyE?si=Z4UiLERgx13Ye7vn')
video

# Let's try online

In [ ]:
from IPython.display import IFrame

url = "https://setosa.io/ev/principal-component-analysis/"

iframe = IFrame(src=url, width="100%", height=300)
iframe


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mticker
import seaborn as sns

# Using `scikit-learn` PCA

In [5]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Variância dos dados originais

In [14]:
print('\nVariância de cada atributo:\n', np.var(df.drop('species',axis=1), axis=0).values)
print('\nVariância acumulada:\n', np.var(df.drop('species',axis=1), axis=0).values / np.var(df.drop('species',axis=1), axis=0).values.sum())



Variância de cada atributo:
 [0.68112222 0.18871289 3.09550267 0.57713289]

Variância acumulada:
 [0.14994532 0.04154411 0.68145793 0.12705264]


## Dados transformados

In [20]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df.drop('species', axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_scaled)

print('\nNovos valores:\n', X_pca[0:5,:], '\n(...)')
print('\nVariância de cada Componente (os valores transformados):\n', pca.explained_variance_)
print('\nVariância acumulada:\n', pca.explained_variance_ratio_)



Novos valores:
 [[-2.26470281  0.4800266   0.12770602 -0.0241682 ]
 [-2.08096115 -0.67413356  0.23460885 -0.10300677]
 [-2.36422905 -0.34190802 -0.04420148 -0.02837705]
 [-2.29938422 -0.59739451 -0.09129011  0.06595556]
 [-2.38984217  0.64683538 -0.0157382   0.03592281]] 
(...)

Variância de cada Componente (os valores transformados):
 [2.93808505 0.9201649  0.14774182 0.02085386]

Variância acumulada:
 [0.72962445 0.22850762 0.03668922 0.00517871]


## PCA Clustering


### Empregando apenas 2 componentes principais ou min 80%  da variância acumulada.

In [35]:
from sklearn.cluster import KMeans

X = df.drop('species', axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2) # pca = PCA(0.8)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=3, random_state=1,n_init="auto").fit(X_pca)

clusters = pd.DataFrame()
clusters['kmeans_pca'] = kmeans.labels_
display(clusters.iloc[np.random.choice(clusters.index,5)])
display(clusters.groupby('kmeans_pca').kmeans_pca.count())

,kmeans_pca
11,1
73,0
113,0
103,2
65,2


,kmeans_pca
kmeans_pca,
0,46
1,49
2,55


## Comparando com Clustering sem PCA

O quando que essa clusterização com apenas 2 componentes se assemelha a clusterização empregando todos os atributos originais?

In [36]:
kmeans = KMeans(n_clusters=3, random_state=1,n_init="auto").fit(X_scaled)

clusters['kmeans_original'] = kmeans.labels_
display(clusters.iloc[np.random.choice(clusters.index,5)])
display(clusters.groupby('kmeans_original').kmeans_pca.count())

,kmeans_pca,kmeans_original
99,0,0
122,2,2
96,0,0
57,0,0
31,1,1


,kmeans_pca
kmeans_original,
0,46
1,49
2,55


In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(clusters['kmeans_original'],clusters['kmeans_pca']))
print('\nPercentual de coincidência:\n', accuracy_score(clusters['kmeans_original'],clusters['kmeans_pca']))


[[46  0  0]
 [ 0 49  0]
 [ 0  0 55]]

Percentual de coincidência:
 1.0


## PCA Classification


### Empregando apenas 2 componentes principais ou min 80%  da variância acumulada.

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df.drop(columns='species')
y = df.species

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=3)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pca = PCA(n_components=2) # pca = PCA(0.8)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

clf = LogisticRegression(max_iter=100)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

clf.score(X_test,y_test)

0.9111111111111111

## Classificação sem PCA

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df.drop(columns='species')
y = df.species

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=3)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = LogisticRegression(max_iter=100)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

clf.score(X_test,y_test)

0.9111111111111111

# Exercício

a) Faça um modelo de classificação dos dados a seguir com uma `RandomForest`. Compare os resultados do modelo empregando todos os dados de entrada e empregando somente dados que correspondem a mais de 70% da variância dos dados.

Empregue para separação dos conjuntos treinamento e teste, 0.3 de teste estratificados e `random_state=1` em todos os casos (lembre-se do `RandomForest`).

b) Faça um gráfico PC1 $\times$ PC2 e verifique isso proporciona alguma separação dos casos benignos e malignos. (dica: empregue cores diferentes para os pontos benignos e malignos).



In [ ]:
df = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/breast-cancer-wisconsin.csv')
df = df[df.columns[0:-1]]
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
